# State Employee Credit Card Transactions — Phase 3: Data Cleaning

**Project Summary:** This project analyzes state employee credit card transactions to identify spending patterns, high-value or unusual transactions, and potential data quality issues.  
**This Notebook (Phase 3) Purpose:** Clean and standardize the raw dataset to produce an analysis-ready table for dashboards and further modeling.

**Objectives of Phase 3**
- Normalize text fields (departments, divisions, merchants, categories).
- Parse and validate dates and numeric types.
- Create transaction flags (e.g., refunds, high-value, potential duplicates).
- Export a clean table for Tableau.

**Deliverables**
- `clean_state_cc_phase3.csv` (analysis-ready)
- A short data dictionary for new/changed columns
- Inline “before → after” checks for each cleaning step (transparency and auditability)

**Scope & Assumptions**
- Only transactional fields included in the raw file are cleaned here.
- No external enrichment (merchant master, GL codes) in this phase.
- Duplicates are flagged (not hard-deleted) to preserve auditability.

## 1. Data Loading and Preview

The dataset **State Employee Credit Card Transactions** was loaded into a pandas DataFrame from a CSV file.  
Warnings were suppressed for cleaner output. The first five rows were displayed to verify successful loading and review the structure.

In [1]:
# Load dataset and inspect the first 5 rows
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Read dataset
df = pd.read_csv("C:/Users/theha/OneDrive/Desktop/Projects/2. State Employee Credit Card Transaction/Data_raw/State_Employee_Credit_Card_Transactions_20250804.csv", parse_dates=["TRANS_DT"], na_values=["", " ", "N/A", "NULL"])

# Screening of data structure
df.head()

,FISCAL_YEAR,FISCAL_PERIOD,DEPT_NAME,DIV_NAME,MERCHANT,CAT_DESCR,TRANS_DT,MERCHANDISE_AMT
0,2025,12,FREIRE CHARTER SCHOOL,Freire Charter School,AMAZON MKTPL*N64SF7SY0,Book Stores,2025-06-06,299.70
1,2025,12,FREIRE CHARTER SCHOOL,Freire Charter School,CITY OF WILMINGTON,Utlts-Elctrc Gas Heating Oil Sanitary Water,2025-06-13,792.48
2,2025,12,FREIRE CHARTER SCHOOL,Freire Charter School,BJS WHOLESALE #0354,Wholesale Clubs,2025-05-27,55.94
3,2025,12,FREIRE CHARTER SCHOOL,Freire Charter School,CHICK-FIL-A #05288,Fast Food Restaurants,2025-06-04,549.00
4,2025,12,FREIRE CHARTER SCHOOL,Freire Charter School,DISCOUNTMUGS.COM,Miscellaneous General Merchandise,2025-05-30,3392.60


## 1.1) Dataset structure & missing value check

Before starting the cleaning process, we inspect the dataset to understand:
- **Shape** — number of rows and columns.
- **Data types** — ensure numerical/date fields are correctly parsed.
- **Missing values** — identify if any fields require imputation or removal.

In [2]:
print(df.shape)
print(df.dtypes)

(742191, 8)
FISCAL_YEAR                 int64
FISCAL_PERIOD               int64
DEPT_NAME                  object
DIV_NAME                   object
MERCHANT                   object
CAT_DESCR                  object
TRANS_DT           datetime64[ns]
MERCHANDISE_AMT           float64
dtype: object


In [3]:
print(df.isna().sum())

FISCAL_YEAR        0
FISCAL_PERIOD      0
DEPT_NAME          0
DIV_NAME           0
MERCHANT           0
CAT_DESCR          0
TRANS_DT           0
MERCHANDISE_AMT    0
dtype: int64


**Observations from this dataset:**
- Shape: **742,191 rows × 8 columns**.
- `FISCAL_YEAR`, `FISCAL_PERIOD` are integers.
- `DEPT_NAME`, `DIV_NAME`, `MERCHANT`, `CAT_DESCR` are text (object) — will need normalization.
- `TRANS_DT` is already in `datetime64[ns]` format.
- `MERCHANDISE_AMT` is `float64` — correct for monetary values.
- No missing values in any column.

## 1.2) Duplicate check using a composite key

Duplicate transactions can occur when:
- The same purchase is recorded twice (e.g., system glitches).
- Multiple systems feed the same transaction into the dataset.
- Two legitimate transactions happen to have identical details (e.g., split bills, recurring charges).

**Method:**  
We define a **composite key** — a set of columns that together should uniquely identify a transaction:
- `DEPT_NAME`, `MERCHANT`, `CAT_DESCR`, `TRANS_DT`, `MERCHANDISE_AMT`.

Using `DataFrame.duplicated(subset=key_cols)`, we count rows where all these fields match another row.

In [4]:
# Duplicate check using composite key
key_cols = ["DEPT_NAME", "MERCHANT", "CAT_DESCR", "TRANS_DT", "MERCHANDISE_AMT"]
print("Duplicate rows:", df.duplicated(subset=key_cols).sum())

Duplicate rows: 50685


**Results:**  
- Found **50,685** duplicate rows based on the composite key.
- These are not removed immediately — instead, we will flag them in later steps for transparency and auditability.

### 1.3 Merchandise Amount Analysis

The `MERCHANDISE_AMT` column was analyzed to understand the distribution of transaction amounts.

In [5]:
# Amount column stats
print(df["MERCHANDISE_AMT"].describe())
print("Negative amounts:", (df["MERCHANDISE_AMT"] < 0).sum())

count    742191.000000
mean        449.331251
std        2551.582215
min     -368335.100000
25%          23.990000
50%          75.920000
75%         281.250000
max      379505.580000
Name: MERCHANDISE_AMT, dtype: float64
Negative amounts: 15364


**Notable Findings:**
- 15,364 transactions have negative amounts, likely indicating refunds or adjustments.
- The large range between minimum and maximum values suggests the presence of outliers.


### 1.4 Transaction Date Range

The `TRANS_DT` column was examined to determine the time span of the dataset.

In [6]:
# Date column stats
print("Min date:", df["TRANS_DT"].min())
print("Max date:", df["TRANS_DT"].max())

Min date: 2021-06-09 00:00:00
Max date: 2025-06-27 00:00:00


**Findings:**
- Earliest transaction date: 2021-06-09
- Latest transaction date: 2025-06-27

This indicates the dataset covers approximately four years of transaction history.

## 2. Data Cleaning and Preparation

Moving forward from the initial dataset overview, the next step focuses on **data cleaning** to ensure consistency and accuracy in subsequent analysis.

### 2.1 Text Normalization
Standardize categorical text fields to remove inconsistencies caused by spacing and capitalization issues.

**Columns processed:**
- `DEPT_NAME`
- `DIV_NAME`
- `MERCHANT`
- `CAT_DESCR`

**Cleaning steps:**
1. Trim and collapse multiple spaces into one.
2. Preserve original casing for `MERCHANT` (vendor IDs).
3. Apply title case to other categorical fields.
4. Compare unique value counts before and after cleaning.

In [7]:
# Step 3.1 – Text cleaning

cat_cols = ["DEPT_NAME", "DIV_NAME", "MERCHANT", "CAT_DESCR"]

def normalize_text(s):
    if pd.isna(s): 
        return s
    return " ".join(str(s).strip().split())

# keep original case for MERCHANT; title-case others
for c in cat_cols:
    df[c] = df[c].map(normalize_text)

for c in ["DEPT_NAME","DIV_NAME","CAT_DESCR"]:
    df[c] = df[c].str.title()
    
# Check changes
for c in cat_cols:
    print(f"{c} unique before cleaning:", df[c].nunique())
    print(df[c].value_counts().head(5))
    print("-"*40)

DEPT_NAME unique before cleaning: 67
DEPT_NAME
Dept Of Natrl Res And Env Cont    93140
Dept Of Corrections               76753
Dept Of Transportation            66748
Appoquinimink School District     47813
Dept Of Health And Social Sv      47413
Name: count, dtype: int64
----------------------------------------
DIV_NAME unique before cleaning: 304
DIV_NAME
Appoquinimink School District    47813
Maintenance Districts            43687
Parks And Recreation             43016
Fish And Wildlife                22119
Facilities Maintenance           18392
Name: count, dtype: int64
----------------------------------------
MERCHANT unique before cleaning: 183141
MERCHANT
GRAINGER                       15651
VERIZONWRLSS*RTCCR VB          11901
EASTERN SHORE COFFEE And WA     8024
UBER TRIP                       5265
WALMART.COM                     4881
Name: count, dtype: int64
----------------------------------------
CAT_DESCR unique before cleaning: 252
CAT_DESCR
Book Stores                  

**Results:**
- Reduced inconsistencies in department, division, and category descriptions.
- Preserved vendor formatting in `MERCHANT` to retain unique IDs.
- Example: `"Dept of Corrections"` and `"DEPT OF CORRECTIONS"` are now standardized.

### 2.2 Amount Flags and Thresholds
Create additional features for the `MERCHANDISE_AMT` column to support outlier detection and financial anomaly analysis.

**Steps:**
1. `IS_REFUND` – flag transactions with negative amounts.
2. `AMOUNT_ABS` – store absolute transaction value for consistent analysis.
3. `IS_HIGH_VALUE` – flag transactions exceeding the 99.5th percentile within each department.

In [8]:
amt_col = "MERCHANDISE_AMT"

# Refund flag
df["IS_REFUND"] = (df[amt_col] < 0).astype("int8")

# Absolute amount
df["AMOUNT_ABS"] = df[amt_col].abs()

# High-value threshold per department
thr = df.groupby("DEPT_NAME")[amt_col].transform(lambda x: x.quantile(0.995))
df["IS_HIGH_VALUE"] = (df["AMOUNT_ABS"] > thr).astype("int8")

# Quick check
print("Refund count:", df["IS_REFUND"].sum())
print("High-value count:", df["IS_HIGH_VALUE"].sum())
print(df[[amt_col,"IS_REFUND","IS_HIGH_VALUE"]].describe())

Refund count: 15364
High-value count: 3773
       MERCHANDISE_AMT      IS_REFUND  IS_HIGH_VALUE
count    742191.000000  742191.000000  742191.000000
mean        449.331251       0.020701       0.005084
std        2551.582215       0.142381       0.071118
min     -368335.100000       0.000000       0.000000
25%          23.990000       0.000000       0.000000
50%          75.920000       0.000000       0.000000
75%         281.250000       0.000000       0.000000
max      379505.580000       1.000000       1.000000


**Results:**
- Refund transactions: **15,364**
- High-value transactions: **3,773**

**Insights:**
- Refund flag helps isolate reversed or corrected charges.
- High-value flag supports identification of potential anomalies for further investigation.

### 2.3 Duplicate Transaction Check

Identify and review potential duplicate transactions to ensure data integrity.

**Steps:**
1. Define key columns for duplicate detection:  
   `FISCAL_YEAR`, `FISCAL_PERIOD`, `DEPT_NAME`, `DIV_NAME`, `MERCHANT`, `CAT_DESCR`, `TRANS_DT`, `MERCHANDISE_AMT`
2. Use `duplicated()` to find repeated rows based on these keys.
3. Group duplicates for inspection by counting occurrences.
4. Display samples of duplicate groups for manual verification.

Purpose:  
- Detect possible repeated transactions that could distort analysis.
- Support decision-making on whether to keep or remove them.

In [9]:
# Step 3.4 – Exact-duplicate flag (keep rows)

key_cols = ["DEPT_NAME","MERCHANT","CAT_DESCR","TRANS_DT","MERCHANDISE_AMT"]

# 1) Size of each key-group
grp_size = df.groupby(key_cols, dropna=False).size().rename("KEY_GROUP_SIZE")
df = df.merge(grp_size.reset_index(), on=key_cols, how="left")

# 2) Flag duplicates (any group with size > 1)
df["IS_EXACT_DUP"] = (df["KEY_GROUP_SIZE"] > 1).astype("int8")

# 3) Quick stats
dup_rows = int(df["IS_EXACT_DUP"].sum())
dup_groups = int(df.loc[df["IS_EXACT_DUP"].eq(1), key_cols].drop_duplicates().shape[0])
print(f"Rows flagged as exact duplicates: {dup_rows}")
print(f"Distinct duplicate groups: {dup_groups}")

# 4) Inspect a few duplicate groups (top by size)
ex = (
    df[df["IS_EXACT_DUP"]==1]
    .sort_values("KEY_GROUP_SIZE", ascending=False)
    .loc[:, key_cols + ["KEY_GROUP_SIZE"]]
    .drop_duplicates()
    .head(10)
)
print("\nSample duplicate key combos (top 10 by group size):")
print(ex)

# (Optional) Keep a stable example table to eyeball the actual rows
sample_keys = ex.head(3)[key_cols]  # first 3 groups
print("\nPreview rows from first duplicate group:")
mask = (df[key_cols] == sample_keys.iloc[0]).all(axis=1)
print(df.loc[mask].head(5))

Rows flagged as exact duplicates: 76675
Distinct duplicate groups: 25990

Sample duplicate key combos (top 10 by group size):
                             DEPT_NAME                  MERCHANT  \
4558         Del Tech And Comm College   DLTCRP BACKGROUND CHECK   
327672       Del Tech And Comm College   DLTCRP BACKGROUND CHECK   
182990       Del Tech And Comm College   DLTCRP BACKGROUND CHECK   
297928       Del Tech And Comm College             COL PRKNG WTC   
378470   Caesar Rodney School District    HOLIDAY INN EXP DENVER   
195302   New Castle County Vo-Tech Sch   DLTCRP BACKGROUND CHECK   
518332       Del Tech And Comm College   DLTCRP BACKGROUND CHECK   
581612       Del Tech And Comm College   DLTCRP BACKGROUND CHECK   
403860  Sussex Technical Schl District   DLTCRP BACKGROUND CHECK   
356518                       Executive  CITY OF WILM-DIV REVENUE   

                                           CAT_DESCR   TRANS_DT  \
4558    Government Services-Not Elsewhere Classified 2025-

**Results:**
- Duplicate transactions were found, with some groups repeating multiple times.
- Example: certain merchants like *Government Services via Payphone* appear multiple times with identical amounts and dates.
- Further investigation is required to determine if these are true duplicates or legitimate repeated purchases.

### 2.4 Deriving Calendar Features

Extract additional date-related fields from `TRANS_DT` to enable time-based analysis.

**Steps:**
1. `TX_DATE` – transaction date (no time component).
2. `TX_YEAR` – transaction year.
3. `TX_MONTH` – transaction month (numeric).
4. `DAY_OF_WEEK` – day name (e.g., Monday, Tuesday).
5. `IS_WEEKEND` – flag indicating weekend transactions (Saturday or Sunday).

Purpose:  
- Enhance temporal analysis by allowing grouping and filtering by year, month, weekday, or weekend activity.

In [10]:
# Derive calendar parts
df["TX_DATE"] = df["TRANS_DT"].dt.date
df["TX_YEAR"] = df["TRANS_DT"].dt.year
df["TX_MONTH"] = df["TRANS_DT"].dt.month
df["DAY_OF_WEEK"] = df["TRANS_DT"].dt.day_name()
df["IS_WEEKEND"] = df["DAY_OF_WEEK"].isin(["Saturday","Sunday"]).astype("int8")

**Results:**
- Calendar-based features successfully added, enabling richer trend and pattern analysis.
- Weekend transactions are now easily identifiable for behavior comparison.

### 2.5 Handling Missing Values

Address potential missing data in categorical and date fields to ensure completeness.

**Steps:**
1. For `DEPT_NAME`, `DIV_NAME`, `MERCHANT`, and `CAT_DESCR`:
   - Create a flag column `IS_UNKNOWN_<col>` if any values are missing.
   - Fill missing values with `"Unknown"`.

2. For `TRANS_DT`:
   - Drop rows where the transaction date is missing.
   - Log the number of rows removed.

Purpose:  
- Prevent issues during grouping or analysis caused by null values.
- Maintain transparency by flagging missing records instead of silently removing them.

In [11]:
# If any new blanks appear later, fill + flag
for c in ["DEPT_NAME","DIV_NAME","MERCHANT","CAT_DESCR"]:
    miss = df[c].isna().sum()
    df[f"IS_UNKNOWN_{c}"] = df[c].isna().astype("int8")
    if miss:
        df[c] = df[c].fillna("Unknown")

# Dates: if any nulls ever occur, we’ll drop & log them
rows_before = len(df)
df = df[ df["TRANS_DT"].notna() ].copy()
rows_dropped_dates = rows_before - len(df)
print("Rows dropped due to missing TRANS_DT:", rows_dropped_dates)

Rows dropped due to missing TRANS_DT: 0


**Results:**
- No missing `TRANS_DT` values were found; no rows dropped.
- Any future blanks in key categorical fields will be flagged and filled with `"Unknown"`.

### 2.6 Data Type Optimization

Convert columns to more memory-efficient data types to improve processing performance.

**Steps:**
1. **Categorical conversion** –  
   Convert text-based fields to the `category` type:  
   - `DEPT_NAME`, `DIV_NAME`, `MERCHANT`, `CAT_DESCR`, `DAY_OF_WEEK`

2. **Integer conversion for flags** –  
   Convert binary flag fields to `int8` for minimal memory usage:  
   - `IS_REFUND`, `IS_HIGH_VALUE`, `IS_EXACT_DUP`, `IS_WEEKEND`  
   - `IS_UNKNOWN_*` flag columns

3. **Fiscal fields as category (optional)** –  
   Convert `FISCAL_YEAR` and `FISCAL_PERIOD` to `category`.

Purpose:  
- Reduce memory footprint of the dataset.
- Improve performance for grouping, filtering, and joins.

In [12]:
# Categoricals
for c in ["DEPT_NAME","DIV_NAME","MERCHANT","CAT_DESCR","DAY_OF_WEEK"]:
    df[c] = df[c].astype("category")

# Small ints for flags
for c in ["IS_REFUND","IS_HIGH_VALUE","IS_EXACT_DUP","IS_WEEKEND",
          "IS_UNKNOWN_DEPT_NAME","IS_UNKNOWN_DIV_NAME","IS_UNKNOWN_MERCHANT","IS_UNKNOWN_CAT_DESCR"]:
    if c in df.columns:
        df[c] = df[c].astype("int8")

# Fiscal fields as category (optional)
df["FISCAL_YEAR"] = df["FISCAL_YEAR"].astype("category")
df["FISCAL_PERIOD"] = df["FISCAL_PERIOD"].astype("category")

**Results:**
- Key text fields converted to categories, significantly reducing memory usage.
- All flag columns stored as compact `int8` type.
- Fiscal fields now treated as categories for cleaner grouping and summarization.

### 2.7 Data Validation Checks

Perform validation tests to ensure data consistency and correctness after cleaning.

**Checks performed:**
1. **Type validation**
   - `TRANS_DT` is a datetime field.
   - `MERCHANDISE_AMT` is numeric.

2. **Date sanity**
   - Minimum date is on or after `2020-01-01`.
   - Maximum date does not exceed tomorrow’s date.

3. **Refund logic consistency**
   - All transactions flagged as `IS_REFUND = 1` have negative amounts.

4. **Duplicate flag presence**
   - `IS_EXACT_DUP` column exists to track flagged duplicates.

Purpose:  
- Verify that data transformations preserved integrity.
- Catch potential anomalies before moving to analysis.

In [13]:
# Types
assert pd.api.types.is_datetime64_any_dtype(df["TRANS_DT"])
assert pd.api.types.is_numeric_dtype(df["MERCHANDISE_AMT"])

# Date sanity
assert df["TRANS_DT"].min() >= pd.Timestamp("2020-01-01")
assert df["TRANS_DT"].max() <= pd.Timestamp.today() + pd.Timedelta(days=1)

# Refund logic consistent
assert (df.loc[df["IS_REFUND"]==1, "MERCHANDISE_AMT"] < 0).all()

# Duplicates: we flagged, so duplicates may remain; ensure flag present
assert "IS_EXACT_DUP" in df.columns

**Results:**
- All validation checks passed successfully.
- Dataset is ready for exploratory analysis and visualization.

### 2.8 Exporting Clean Dataset

Prepare and export the cleaned dataset for further analysis and dashboard creation.

**Steps:**
1. Define `cols_order` – list of columns required for dashboards and EDA.
2. Keep only the columns that exist in the DataFrame.
3. Export to:
   - `transactions_clean_for_EDA.csv` – optimized for Tableau dashboard input.
   - `transactions_clean.csv` – full cleaned dataset for general use.

Purpose:  
- Ensure consistent column ordering for downstream tools.
- Provide both a tailored dataset for visualization and a complete cleaned version for other analyses.

In [14]:
# Columns needed for dashboards
cols_order = [
    "FISCAL_YEAR", "FISCAL_PERIOD",
    "DEPT_NAME", "DIV_NAME", "MERCHANT", "CAT_DESCR",
    "TRANS_DT", "TX_DATE", "TX_YEAR", "TX_MONTH", "DAY_OF_WEEK", "IS_WEEKEND",
    "MERCHANDISE_AMT", "AMOUNT_ABS",
    "IS_REFUND", "IS_HIGH_VALUE", "IS_EXACT_DUP"
]

# Keep only the columns that actually exist in df
cols_order = [c for c in cols_order if c in df.columns]

# Export to CSV for Tableau
df[cols_order].to_csv("transactions_clean_for_EDA.csv", index=False, encoding="utf-8-sig")

print("Export complete. Rows exported:", len(df))

Export complete. Rows exported: 742191


In [15]:
# Export clean dataset
df.to_csv("transactions_clean.csv", index=False, encoding="utf-8-sig")

**Results:**
- `transactions_clean_for_EDA.csv` – 742,191 rows exported for dashboard creation.
- `transactions_clean.csv` – full cleaned dataset saved for future analysis.